In [ ]:
import numpy as np
from scipy import stats
import time
import serial
import datetime
import mysql.connector
import math

class Turbidimetro:
    def __init__(self):
        # Dados de calibração
        self.leituras_ldr = [25.73, 35.80, 66.51]
        self.turbidez = [0, 100, 800]

        # Ajuste do modelo de regressão linear
        self.slope, self.intercept, _, _, _ = stats.linregress(self.leituras_ldr, self.turbidez)
        self.GAMMA = 0.7
        self.RL10 = 50

        # Configuração da conexão
        self.config = {
            'user': 'root',
            'password': '',
            'host': '127.0.0.1',
            'database': 'sin499-turbidimetro',
            'raise_on_warnings': True
        }
        self.cnx = mysql.connector.connect(**self.config)
        self.cursor = self.cnx.cursor()

        self.led = 13
        self.ldr = 'A0'
        self.serial_port = serial.Serial('COM3', 9600)

        # Configuração inicial
        self.setup_serial()

    def setup_serial(self):
        self.serial_port.write(('pinMode(' + str(self.led) + ', OUTPUT)').encode())
        self.serial_port.write(('pinMode(' + self.ldr + ', INPUT)').encode())

    def regressao_linear(self, x):
        return max(self.slope * x + self.intercept, 0)

    def read_ldr_and_lux(self):
        self.serial_port.write(self.ldr.encode())
        response = self.serial_port.readline().decode().strip()
        if response.startswith('Valor LDR: '):
            analogValue = int(response.replace('Valor LDR: ', ''))
            voltage = analogValue / 1024. * 5
            denominator = (1 - voltage / 5)
            
            if voltage >= 5 or denominator == 0:
                return analogValue, None

            resistance = 2000 * voltage / denominator
            if resistance == 0:
                return analogValue, None
            
            lux = round(math.pow(self.RL10 * 1e3 * math.pow(10, self.GAMMA) / resistance, (1 / self.GAMMA)), 3)
            return analogValue, lux
        else:
            return None, None

    def insert_into_db(self, valorldr, turbidez_estimada, current_time):
        query = "INSERT INTO turbidimetro (sensoriamento_ldr, turbidez, date_time) VALUES (%s, %s, %s)"
        values = (valorldr, turbidez_estimada, current_time)
        self.cursor.execute(query, values)
        self.cnx.commit()

    def main(self):
        try:
            while True:
                valorldr, lux_value = self.read_ldr_and_lux()
                if lux_value is not None:
                    turbidez_estimada = self.regressao_linear(lux_value)
                    if turbidez_estimada <= 1000:
                        current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        self.insert_into_db(valorldr, turbidez_estimada, current_time)
                        print(f"{current_time} - Valor LDR: {valorldr} | Turbidez Estimada: {turbidez_estimada}")
                        self.serial_port.write(('digitalWrite(' + str(self.led) + ', HIGH)').encode())
                        time.sleep(3)
        except KeyboardInterrupt:
            pass
        finally:
            self.cursor.close()
            self.cnx.close()

if __name__ == "__main__":
    turbidimetro = Turbidimetro()
    turbidimetro.main()
